---
Обход ограничения максимальной глубины запроса
---
Для обхода максимальной глубины запроса к API, получаем промежутки времени, в которые кол-во опубликованых вакансий не
превышает 2000. Далее будут получены данные по этим промежуткам.

(Данные только о вакансиях в сфере IT)

In [15]:
%%time
from data_parsers.hhParser import HhParser
from datetime import datetime, timedelta

parser = HhParser()
cur_date = {
    'from': (datetime.now() - timedelta(days=1)),
    'to': datetime.now()
}
req_param = {
        'specialization':1,
        'per_page':0,
        'date_from':cur_date['from'].strftime('%Y-%m-%dT%H:%M:%S'),
        'date_to':cur_date['to'].strftime('%Y-%m-%dT%H:%M:%S')
    }
data = parser.execute_request(req_param=req_param)
dates = []
total_vacancies_number = 0
while data['found'] != 0:
    while data['found'] > 2000:
        cur_date['from'] += timedelta(hours=1)
        req_param['date_from'] = cur_date['from'].strftime('%Y-%m-%dT%H:%M:%S')
        data = parser.execute_request(req_param=req_param)

    total_vacancies_number += data['found']
    dates.append([cur_date['from'], cur_date['to']])

    cur_date['to'] = cur_date['from']
    cur_date['from'] -= timedelta(days=1)
    req_param['date_from'] = cur_date['from'].strftime('%Y-%m-%dT%H:%M:%S')
    req_param['date_to'] = cur_date['to'].strftime('%Y-%m-%dT%H:%M:%S')
    data = parser.execute_request(req_param=req_param)
print(f'За последний месяц имеется {total_vacancies_number} вакансий')

За последний месяц имеется 119405 вакансий
Wall time: 6min 8s


---
Выгрузка всех вакансий
---
Получение данных по вакансиям за ранее полученные промежутки времени

In [3]:
%%time
from services.db_service import DbService

db_service = DbService()

vacancies = parser.get_vacancies(req_params={'specialization':1})
db_service.save_vacancies(vacancies)

Получение id вакансий:   0%|          | 0/19 [00:00<?, ?it/s]

KeyboardInterrupt: 